In [3]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, matthews_corrcoef, roc_auc_score, roc_curve
import matplotlib.pyplot as plt
import seaborn as sns
from imblearn.over_sampling import SMOTE
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

In [4]:
filename = 'C:\\Users\\InbarBlech\\PycharmProjects\\Thesis\\gene_specific_df\\COL2A1_with_position.csv'
data = pd.read_csv(f"{filename}")
data.head(10)
dot_index = filename.index('_')
gene = filename[:dot_index]
if gene == "combined" or gene == "combined_with_source":
    gene = "7 genes"
if gene == "features":
    gene = "190 genes"
    
data

,gene,variant,pathogenicity,uniprot_id,stability_WT,stability_MUT,blosum,hydrophobicity_WT,hydrophobicity_MUT,volume_WT,...,oda_MUT,sasa_WT,sasa_MUT,RSA_WT,RSA_MUT,oda_delta,sasa_delta,pssm,entropy,position
0,COL2A1,M1R,benign,P02458,3948.38,3948.44,-1,1.9,-4.5,162.9,...,6.46,251.72,271.12,1.240000,1.023094,8.12,19.40,6.659,0.967,1
1,COL2A1,I2A,benign,P02458,3948.28,3948.48,-1,4.5,1.8,166.7,...,-7.46,155.49,92.52,0.797385,0.764628,-0.33,-62.97,4.501,2.775,2
2,COL2A1,I2L,benign,P02458,3948.28,3948.09,2,4.5,3.8,166.7,...,-9.95,155.49,131.32,0.797385,0.687539,-2.82,-24.17,4.501,2.775,2
3,COL2A1,I2P,benign,P02458,3948.28,3950.85,-3,4.5,-1.6,166.7,...,-6.81,155.49,113.71,0.797385,0.738377,0.32,-41.78,4.501,2.775,2
4,COL2A1,I2T,benign,P02458,3948.28,3948.60,-1,4.5,-0.7,166.7,...,-4.41,155.49,110.44,0.797385,0.677546,2.72,-45.05,4.501,2.775,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
946,COL2A1,P952T,benign,NaN,3948.53,3945.75,-1,-1.6,-0.7,112.7,...,-1.02,126.88,119.96,0.823896,0.735951,1.72,-6.92,2.011,2.294,952
947,COL2A1,G1405S,benign,NaN,3949.08,3951.75,0,-0.4,-0.8,60.1,...,2.10,57.32,93.49,0.590928,0.653776,1.81,36.17,2.249,0.847,1405
948,COL2A1,T638I,benign,NaN,3948.05,3947.92,-1,-0.7,4.5,116.1,...,-1.23,145.62,166.36,0.893374,0.853128,-1.79,20.74,5.112,1.220,638
949,COL2A1,E142D,benign,NaN,3948.45,3947.32,2,-3.5,-3.5,138.4,...,3.68,173.18,142.93,0.809252,0.764332,0.36,-30.25,4.092,2.069,142


In [5]:

data = pd.get_dummies(data, columns=["secondary_structure"])

mapping = {"benign": 0, "pathogenic": 1}
data["pathogenicity"] = data["pathogenicity"].map(mapping)

data = data.drop(
    labels=["uniprot_id", "stability_WT", "stability_MUT", "hydrophobicity_WT", "hydrophobicity_MUT", "volume_WT",
            "volume_MUT", "sequence_length", "oda_MUT", "oda_WT", "sasa_WT", "sasa_MUT", "RSA_MUT", "variant", "gene",
            "protein_contain_transmembrane", "is_residue_transmembranal", "aa_WT", "aa_MUT"], axis=1, inplace=False)

data

,pathogenicity,blosum,plddt_residue,stability_delta,hydrophobicity_delta,volume_delta,RSA_WT,oda_delta,sasa_delta,pssm,entropy,position,secondary_structure_Beta strand,secondary_structure_Helix,secondary_structure_Loop,secondary_structure_Turn
0,0,-1,35.42,0.06,-6.4,10.5,1.240000,8.12,19.40,6.659,0.967,1,False,False,True,False
1,0,-1,38.29,0.20,-2.7,-78.1,0.797385,-0.33,-62.97,4.501,2.775,2,False,False,True,False
2,0,2,38.29,-0.19,-0.7,0.0,0.797385,-2.82,-24.17,4.501,2.775,2,False,False,True,False
3,0,-3,38.29,2.57,-6.1,-54.0,0.797385,0.32,-41.78,4.501,2.775,2,False,False,True,False
4,0,-1,38.29,0.32,-5.2,-50.6,0.797385,2.72,-45.05,4.501,2.775,2,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
946,0,-1,47.41,-2.78,0.9,3.4,0.823896,1.72,-6.92,2.011,2.294,952,False,False,True,False
947,0,0,93.77,2.67,-0.4,28.9,0.590928,1.81,36.17,2.249,0.847,1405,False,False,True,False
948,0,-1,42.56,-0.13,5.2,50.6,0.893374,-1.79,20.74,5.112,1.220,638,False,False,True,False
949,0,2,54.93,-1.13,0.0,-27.3,0.809252,0.36,-30.25,4.092,2.069,142,False,False,True,False


In [6]:
structural_features = ["oda_delta", "stability_delta", "secondary_structure_Beta strand", "secondary_structure_Helix", "secondary_structure_Loop", "sasa_delta", "RSA_WT"]
# Drop structural features
data = data.drop(labels=["stability_delta"], axis=1, inplace=False)

In [7]:

# Create list of all positions.
positions = data["position"].unique()
tps = []
fps = []
fns = []
tns = []
errors = []
mistakes = 0

counter = 0

for pos in positions:
    counter += 1
    print(f"Position: {pos} ({counter}/{len(positions)})")
    # Create train and test sets.
    train = data[data["position"] != pos]
    test = data[data["position"] == pos]

    X_test = test.drop(labels=["pathogenicity", "position"], axis=1, inplace=False)
    y_test = test["pathogenicity"]

    # Oversamole the train set using SMOTE.
    X_train = train.drop(labels=["pathogenicity", "position"], axis=1, inplace=False)
    y_train = train["pathogenicity"]
    oversample = SMOTE(sampling_strategy='minority', random_state=42)
    X_train_resampled, y_train_resampled = oversample.fit_resample(X_train, y_train)
    # class_distribution = y_train_resampled.value_counts()
    # print(f"Training set: (SMOTE)\n{class_distribution}")

    xgb_classifier = xgb.XGBClassifier(learning_rate=0.1, max_depth=3, n_estimators=100, random_state=42)
    xgb_classifier.fit(X_train_resampled, y_train_resampled)  # Fit the model with the resampled data
    y_pred = xgb_classifier.predict(X_test)

    tp = sum((y_test == 1) & (y_pred == 1))
    fp = sum((y_test == 0) & (y_pred == 1))
    fn = sum((y_test == 1) & (y_pred == 0))
    tn = sum((y_test == 0) & (y_pred == 0))

    print(f"TP: {tp}, FP: {fp}, TN: {tn}, FN: {fn}")

    tps.append(tp)
    fps.append(fp)
    fns.append(fn)
    tns.append(tn)

    print(f"tps: {sum(tps)}, fps: {sum(fps)}, fns: {sum(fns)}, tns: {sum(tns)}")

    print(f"Prediction: {y_pred}. Reality: {y_test.values}")
    # # print error if at least one prediction is wrong
    # if not np.array_equal(y_pred, y_test.values):
    #     print(f"Classification is wrong for position {pos}!")
    #     for i in range(len(y_pred)):
    #         if y_pred[i] != y_test.values[i]:
    #             mistakes+=1
    #             print(f"Predicted: {y_pred[i]}, reality: {y_test.values[i]}")

# Calculate MCC
TP = sum(tps)
FP = sum(fps)
FN = sum(fns)
TN = sum(tns)
mcc = (TP * TN - FP * FN) / np.sqrt((TP + FP) * (TP + FN) * (TN + FP) * (TN + FN))

print(f"TP: {sum(tps)}, FP: {sum(fps)}, TN: {sum(fns)}, FN: {sum(tns)}")

sensitivity = TP / (TP + FN)
specificity = TN / (TN + FP)
precision = TP / (TP + FP)
accuracy = (TP + TN) / (TP + TN + FP + FN)
print(f"Results for {gene}:")
print(f"Sensitivity (Recall): {sensitivity}")
print(f"Specificity: {specificity}")
print(f"Precision: {precision}")
print(f"MCC for {gene}: {mcc}")
print(f"Accuracy: {accuracy}")

Position: 1 (1/593)
TP: 0, FP: 0, TN: 1, FN: 0
tps: 0, fps: 0, fns: 0, tns: 1
Prediction: [0]. Reality: [0]
Position: 2 (2/593)
TP: 0, FP: 0, TN: 5, FN: 0
tps: 0, fps: 0, fns: 0, tns: 6
Prediction: [0 0 0 0 0]. Reality: [0 0 0 0 0]
Position: 3 (3/593)
TP: 0, FP: 1, TN: 1, FN: 0
tps: 0, fps: 1, fns: 0, tns: 7
Prediction: [1 0]. Reality: [0 0]
Position: 4 (4/593)
TP: 0, FP: 0, TN: 3, FN: 0
tps: 0, fps: 1, fns: 0, tns: 10
Prediction: [0 0 0]. Reality: [0 0 0]
Position: 5 (5/593)
TP: 0, FP: 0, TN: 1, FN: 0
tps: 0, fps: 1, fns: 0, tns: 11
Prediction: [0]. Reality: [0]
Position: 6 (6/593)
TP: 0, FP: 0, TN: 2, FN: 0
tps: 0, fps: 1, fns: 0, tns: 13
Prediction: [0 0]. Reality: [0 0]
Position: 7 (7/593)
TP: 0, FP: 0, TN: 3, FN: 0
tps: 0, fps: 1, fns: 0, tns: 16
Prediction: [0 0 0]. Reality: [0 0 0]
Position: 8 (8/593)
TP: 0, FP: 0, TN: 3, FN: 0
tps: 0, fps: 1, fns: 0, tns: 19
Prediction: [0 0 0]. Reality: [0 0 0]
Position: 9 (9/593)
TP: 0, FP: 0, TN: 4, FN: 0
tps: 0, fps: 1, fns: 0, tns: 23
Pred